## Overlap between red and the holotomographic channel

In [1]:
# Step 1: Import all required libraries
import pandas as pd
import numpy as np
from tifffile import imread, imsave
import cv2
from tkinter import filedialog as fd
import os
from datetime import datetime

# Microscope calibration (microns per pixel)
um_per_pixel = 0.0876  # 87.6 nm per pixel

print("All libraries imported successfully!")
print(f"Using calibration: {um_per_pixel} μm/pixel ({um_per_pixel*1000} nm/pixel)")

All libraries imported successfully!
Using calibration: 0.0876 μm/pixel (87.6 nm/pixel)


In [3]:
# Step 2: File selection (Red channel only)
print("Select the RED channel CSV file:")
red_csv = fd.askopenfilename(title="Select RED channel CSV")
print(f"Selected: {red_csv}")

print("Select the holotomographic TIFF file:")
holo_tif = fd.askopenfilename(title="Select holotomographic TIFF")
print(f"Selected: {holo_tif}")

print("File selection complete!")

Select the RED channel CSV file:
Selected: Z:/Bisal_Halder_turbo/V601-Images-Armin/red_channel/condensates_AIO-gal9-lnp-10min-per-frame2.csv
Select the holotomographic TIFF file:
Selected: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography/07.20.25-gal9-lnp-10min-per-frame2_RI_frame001_deconved_MIP_slices_55-86.tif
File selection complete!


In [4]:
# Step 3: Load and inspect the data
print("Loading data...")

# Load CSV file
df_red = pd.read_csv(red_csv)
print(f"Red channel CSV shape: {df_red.shape}")

# Load holotomographic image
holo_img = imread(holo_tif)
print(f"Holotomographic image shape: {holo_img.shape}")
print(f"Holotomographic image dtype: {holo_img.dtype}")
print(f"Holotomographic image intensity range: [{holo_img.min()}, {holo_img.max()}]")

# Display first few rows of data
print("\nRed channel data preview:")
display(df_red.head())

Loading data...
Red channel CSV shape: (271, 12)
Holotomographic image shape: (1024, 1024)
Holotomographic image dtype: uint16
Holotomographic image intensity range: [51, 10000]

Red channel data preview:


,condensateID,contour_coord,center_x_pxl,center_y_pxl,area_um2,R_nm,mean_intensity,max_intensity,max_location,aspect_ratio,contour_solidity,contour_extent
0,1,"[[660, 1015], [659, 1016], [658, 1017], [658, ...",661.303835,1019.675516,0.773429,496.175310,1109.500000,1946.0,"(661, 1019)",1.111111,0.941667,0.627778
1,2,"[[511, 1011], [511, 1012], [511, 1013], [512, ...",512.000000,1012.500000,0.068445,147.603253,1143.900000,1287.0,"(512, 1012)",0.750000,1.000000,0.416667
2,3,"[[952, 1010], [951, 1011], [950, 1012], [950, ...",954.467700,1014.020672,0.882941,530.140255,747.481013,1303.0,"(954, 1014)",1.111111,0.992308,0.716667
3,4,"[[1021, 1006], [1020, 1007], [1019, 1008], [10...",1020.641026,1010.525641,0.533871,412.233450,581.346154,921.0,"(1021, 1011)",0.700000,0.928571,0.557143
4,5,"[[781, 1003], [780, 1004], [779, 1005], [779, ...",782.891068,1008.413943,1.047209,577.353287,922.619565,1429.0,"(782, 1009)",1.000000,0.950311,0.632231


In [5]:
# Step 4: Create and save thresholded holotomographic mask
print("Creating holotomographic mask...")

# Create binary mask for holotomographic signals (intensity > 800)
holo_mask = (holo_img > 800).astype(np.uint8) * 255

print(f"Holotomographic mask shape: {holo_mask.shape}")
print(f"Pixels above threshold: {np.sum(holo_mask > 0)}")
print(f"Percentage of image above threshold: {np.sum(holo_mask > 0) / holo_mask.size * 100:.2f}%")

# Show mask properties
print(f"Mask unique values: {np.unique(holo_mask)}")

# Save only the version with original image values above threshold in the holotomography folder
holo_thresholded_values = np.where(holo_img > 800, holo_img, 0)

# Save in the same folder as the input holotomographic image
holo_dir = os.path.dirname(holo_tif)
thresholded_values_filename = os.path.join(holo_dir, "holo_values_above_800.tif")
imsave(thresholded_values_filename, holo_thresholded_values)
print(f"✓ Holotomographic values above 800 saved to: {thresholded_values_filename}")

print("\nYou can now open this TIFF file in ImageJ to verify the thresholding results:")
print(f"1. {thresholded_values_filename} - Original intensity values (0 for values ≤800)")

Creating holotomographic mask...
Holotomographic mask shape: (1024, 1024)
Pixels above threshold: 67333
Percentage of image above threshold: 6.42%
Mask unique values: [  0 255]
✓ Holotomographic values above 800 saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\holo_values_above_800.tif

You can now open this TIFF file in ImageJ to verify the thresholding results:
1. Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\holo_values_above_800.tif - Original intensity values (0 for values ≤800)


C:\Users\bisal\AppData\Local\Temp\4\ipykernel_11508\1417528937.py:20: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(thresholded_values_filename, holo_thresholded_values)


In [6]:
# Step 5: Define helper functions for red channel only
def count_overlapping_condensates(df, holo_mask, channel_name):
    """Count how many condensates overlap with holotomographic signals"""
    total_condensates = len(df)
    overlapping_condensates = 0
    individual_results = []
    
    print(f"\nProcessing {channel_name} channel: {total_condensates} condensates")
    
    for idx, row in df.iterrows():
        try:
            # Recreate condensate mask from contour coordinates
            contour_coords = eval(row['contour_coord'])
            contour_array = np.array(contour_coords, dtype=np.int32).reshape(-1, 1, 2)
            
            # Create mask for this condensate
            condensate_mask = np.zeros(holo_mask.shape, dtype=np.uint8)
            cv2.fillPoly(condensate_mask, [contour_array], 255)
            
            # Calculate overlap with holo signals
            overlap_mask = cv2.bitwise_and(condensate_mask, holo_mask)
            overlap_pixels = np.sum(overlap_mask > 0)
            
            # Check if this condensate overlaps with holo signals (at least 1 pixel overlap)
            does_overlap = overlap_pixels > 0
            
            if does_overlap:
                overlapping_condensates += 1
            
            individual_results.append({
                'condensate_id': row['condensateID'],
                'channel': channel_name,
                'overlaps_with_holo': does_overlap,
                'overlap_pixels': overlap_pixels,
                'area_um2': row['area_um2'],
                'R_nm': row['R_nm']
            })
            
            if idx < 5:  # Show first 5 for debugging
                status = "OVERLAPS" if does_overlap else "no overlap"
                print(f"  Condensate {idx+1}: {status} ({overlap_pixels} overlap pixels)")
                
        except Exception as e:
            print(f"Error processing {channel_name} condensate {idx+1}: {e}")
            continue
    
    if total_condensates > 0:
        overlap_percentage = (overlapping_condensates / total_condensates) * 100
    else:
        overlap_percentage = 0
        
    print(f"  Total {channel_name} condensates: {total_condensates}")
    print(f"  Overlapping with holo signals: {overlapping_condensates}")
    print(f"  Overlap percentage: {overlap_percentage:.2f}%")
        
    return overlap_percentage, overlapping_condensates, total_condensates, individual_results

print("Helper functions defined!")

Helper functions defined!


In [7]:
# Step 6: Calculate overlaps for red channel only
print("Starting overlap calculations...")
print("="*50)

# Count overlapping condensates for red channel
red_overlap_pct, red_overlap_count, red_total_count, red_individual = count_overlapping_condensates(df_red, holo_mask, "RED")

print("\n" + "="*50)
print("OVERLAP ANALYSIS RESULTS")
print("="*50)
print(f"RED channel: {red_overlap_pct:.2f}% ({red_overlap_count}/{red_total_count}) overlap with holo signals")

Starting overlap calculations...

Processing RED channel: 271 condensates
  Condensate 1: no overlap (0 overlap pixels)
  Condensate 2: no overlap (0 overlap pixels)
  Condensate 3: no overlap (0 overlap pixels)
  Condensate 4: no overlap (0 overlap pixels)
  Condensate 5: no overlap (0 overlap pixels)
  Total RED condensates: 271
  Overlapping with holo signals: 185
  Overlap percentage: 68.27%

OVERLAP ANALYSIS RESULTS
RED channel: 68.27% (185/271) overlap with holo signals


In [8]:
# Step 7: Create and save results + visualization
print("Creating results dataframes and visualization...")

# Create summary results
summary_data = {
    'analysis_type': ['Red Channel Only'],
    'overlap_percentage': [red_overlap_pct],
    'overlapping_condensates': [red_overlap_count],
    'total_condensates': [red_total_count],
    'holo_threshold': [800],
    'um_per_pixel': [um_per_pixel],
    'timestamp': [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
}

summary_df = pd.DataFrame(summary_data)

# Create detailed individual results
individual_df = pd.DataFrame(red_individual)

print("Summary dataframe:")
display(summary_df)
print(f"\nDetailed dataframe shape: {individual_df.shape}")
print("Detailed dataframe preview:")
display(individual_df.head())

# Create visualization TIFF files - Save in holotomography folder
print("\n" + "="*50)
print("CREATING VISUALIZATION TIFF FILES")
print("="*50)

def create_red_holo_visualization(df_red, holo_img, holo_mask, output_dir):
    # Red + Holo overlap visualization
    red_holo_vis = np.zeros((holo_img.shape[0], holo_img.shape[1], 3), dtype=np.uint8)
    
    # Add holotomography in blue
    red_holo_vis[:, :, 2] = (holo_mask > 0).astype(np.uint8) * 255
    
    # Add red condensates
    for idx, row in df_red.iterrows():
        try:
            contour_coords = eval(row['contour_coord'])
            contour_array = np.array(contour_coords, dtype=np.int32).reshape(-1, 1, 2)
            cv2.fillPoly(red_holo_vis, [contour_array], (255, 0, 0))  # Red only
        except Exception as e:
            continue
    
    red_holo_filename = os.path.join(output_dir, "red_holo_overlap.tif")
    imsave(red_holo_filename, red_holo_vis)
    print(f"✓ Red + Holo overlap saved to: {red_holo_filename}")
    
    return red_holo_filename

# Use holotomography folder for all outputs
holo_dir = os.path.dirname(holo_tif)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save CSV files in holotomography folder
summary_filename = os.path.join(holo_dir, f"red_only_overlap_summary_{timestamp}.csv")
summary_df.to_csv(summary_filename, index=False)
print(f"✓ Summary results saved to: {summary_filename}")

individual_filename = os.path.join(holo_dir, f"red_only_overlap_detailed_{timestamp}.csv")
individual_df.to_csv(individual_filename, index=False)
print(f"✓ Detailed results saved to: {individual_filename}")

# Create visualization file in holotomography folder
red_holo_file = create_red_holo_visualization(df_red, holo_img, holo_mask, holo_dir)

print("\n" + "="*50)
print("ANALYSIS COMPLETE!")
print("="*50)
print(f"Red channel: {red_overlap_pct:.2f}% ({red_overlap_count}/{red_total_count}) overlap with holo signals")
print(f"\nTotal red condensates analyzed: {red_total_count}")
print(f"Holo signal threshold: >800 intensity")
print(f"Pixel calibration: {um_per_pixel} μm/pixel ({um_per_pixel*1000} nm/pixel)")
print(f"\nAll files saved to: {holo_dir}")
print(f"\nFiles created:")
print(f"1. {os.path.basename(summary_filename)} - Summary results")
print(f"2. {os.path.basename(individual_filename)} - Detailed condensate data")
print(f"3. red_holo_overlap.tif - Visualization of red + holotomography overlap")
print(f"4. holo_values_above_800.tif - Holotomography threshold verification")

Creating results dataframes and visualization...
Summary dataframe:


,analysis_type,overlap_percentage,overlapping_condensates,total_condensates,holo_threshold,um_per_pixel,timestamp
0,Red Channel Only,68.265683,185,271,800,0.0876,2025-10-14 16:31:59



Detailed dataframe shape: (271, 6)
Detailed dataframe preview:


,condensate_id,channel,overlaps_with_holo,overlap_pixels,area_um2,R_nm
0,1,RED,False,0,0.773429,496.175310
1,2,RED,False,0,0.068445,147.603253
2,3,RED,False,0,0.882941,530.140255
3,4,RED,False,0,0.533871,412.233450
4,5,RED,False,0,1.047209,577.353287



CREATING VISUALIZATION TIFF FILES
✓ Summary results saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\red_only_overlap_summary_20251014_163159.csv
✓ Detailed results saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\red_only_overlap_detailed_20251014_163159.csv
✓ Red + Holo overlap saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\red_holo_overlap.tif

ANALYSIS COMPLETE!
Red channel: 68.27% (185/271) overlap with holo signals

Total red condensates analyzed: 271
Holo signal threshold: >800 intensity
Pixel calibration: 0.0876 μm/pixel (87.6 nm/pixel)

All files saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography

Files created:
1. red_only_overlap_summary_20251014_163159.csv - Summary results
2. red_only_overlap_detailed_20251014_163159.csv - Detailed condensate data
3. red_holo_overlap.tif - Visualization of red + holotomography overlap
4. holo_values_above_800.tif - Holotomography threshold verification


C:\Users\bisal\AppData\Local\Temp\4\ipykernel_11508\1568613666.py:48: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(red_holo_filename, red_holo_vis)
